In [ ]:
using BenchmarkTools
using BoundingVolumeHierarchies
using LinearAlgebra
using StatsBase
using FileIO
using Plots
using Images
import BoundingVolumeHierarchies: Segment, intersection
import GeometryTypes: HomogenousMesh
import BoundingVolumeHierarchies: PointT, _normal
import WGLMakie as Mke
using Meshes, MeshViz
mesh1 = load("04.obj")

In [ ]:
max_x=-Inf
min_x= Inf
max_y=-Inf
min_y= Inf
max_z=-Inf
min_z= Inf

for i=1:size(mesh1)[1]
    for j=1:3
        if mesh1[i][j][1]>max_x
            max_x=mesh1[i][j][1]
        end
        if mesh1[i][j][1]<min_x
            min_x=mesh1[i][j][1]
        end
        if mesh1[i][j][2]>max_y
             max_y=mesh1[i][j][2]
        end
        if mesh1[i][j][2]<min_y
            min_y=mesh1[i][j][2]
        end
        if mesh1[i][j][3]>max_z
            max_z=mesh1[i][j][3]
        end
        if mesh1[i][j][3]<min_z
            min_z=mesh1[i][j][3]
        end
    end
end
print((max_x,max_y,max_z))
print((min_x,min_y,min_z))

In [ ]:
start=Point3([min_x-0.1,min_y-0.1,min_z-0.1])
finish=Point3([max_x+0.1,max_y+0.1,max_z+0.1])
grid=CartesianGrid(start, finish, dims=(round(Int,(max_x-min_x)/0.1),round(Int,(max_y-min_y)/0.1),round(Int,(max_z-min_z)/0.1)))

In [ ]:
size(grid)

In [ ]:
#create point sets and face sets to build the BVH
rverts = [[0.0,0.0,0.0]]
rfaces = [[0,0,0]]
for i=1:5375
    for j=1:3
        push!(rverts,mesh1[i][j])
    end
    connec1=[[3i-2,3i-1,3i]]
    append!(rfaces,connec1)    
end

popfirst!(rfaces)
popfirst!(rverts)

#create BVH
normals = [_normal(rverts[f]) for f in rfaces] # mesh cell normals
rBVH = @time buildBVH(rverts, rfaces)
size1=size(grid)[1]*size(grid)[2]*size(grid)[3]

In [ ]:
global segm = Array{Segment}(undef, 12)
global count=0
intersectionlist = Any[]
for i=1:size1
    segm[1] = Segment(coordinates(Meshes.vertices(grid[i])[1]), coordinates(Meshes.vertices(grid[i])[2]))
    segm[2] = Segment(coordinates(Meshes.vertices(grid[i])[2]), coordinates(Meshes.vertices(grid[i])[3]))
    segm[3] = Segment(coordinates(Meshes.vertices(grid[i])[3]), coordinates(Meshes.vertices(grid[i])[4]))
    segm[4] = Segment(coordinates(Meshes.vertices(grid[i])[4]), coordinates(Meshes.vertices(grid[i])[1]))
    segm[5] = Segment(coordinates(Meshes.vertices(grid[i])[5]), coordinates(Meshes.vertices(grid[i])[6]))
    segm[6] = Segment(coordinates(Meshes.vertices(grid[i])[6]), coordinates(Meshes.vertices(grid[i])[7]))
    segm[7] = Segment(coordinates(Meshes.vertices(grid[i])[7]), coordinates(Meshes.vertices(grid[i])[8]))
    segm[8] = Segment(coordinates(Meshes.vertices(grid[i])[8]), coordinates(Meshes.vertices(grid[i])[1]))
    segm[9] = Segment(coordinates(Meshes.vertices(grid[i])[1]), coordinates(Meshes.vertices(grid[i])[5]))
    segm[10] = Segment(coordinates(Meshes.vertices(grid[i])[2]), coordinates(Meshes.vertices(grid[i])[6]))
    segm[11] = Segment(coordinates(Meshes.vertices(grid[i])[3]), coordinates(Meshes.vertices(grid[i])[7]))
    segm[12] = Segment(coordinates(Meshes.vertices(grid[i])[4]), coordinates(Meshes.vertices(grid[i])[8]))
    if(any(intersection(segm[i], rBVH, rverts, rfaces, normals) for i=1:12))
        count+=1
        push!(intersectionlist,i)
    end
end
    

In [ ]:
walls = Any[x for x in 1:length(intersectionlist)]
for i=1:length(intersectionlist)
    walls[i]=grid[intersectionlist[i]]
end
walllength=length(walls)

f=viz(walls[1])
for x in walls[2:walllength]
    viz!(x)
end

In [ ]:
display(f)

In [ ]:
fulllist = 1:size1
remaininglist = setdiff(fulllist, intersectionlist)

air = Any[x for x in 1:length(remaininglist)]
for i=1:length(remaininglist)
    air[i]=grid[remaininglist[i]]
end
airlength=length(air)

g=viz(air[1])
for x in air[2:airlength]
    viz!(x)
end

In [ ]:
display(g)

In [ ]:
#create point sets and connection sets to form the mesh
points = Point3[]
connec = connect.([(0,0,0)], Ngon)

for i=1:5375
    for j=1:3
        push!(points,Point3(mesh1[i][j]))
    end
    A=(3i-2,3i-1,3i)
    connect.([A],Ngon)
    append!(connec,connect.([A],Ngon))    
end

popfirst!(connec)

# the processed mesh
processed_mesh = SimpleMesh(points, connec)

viz(processed_mesh, showfacets = true)

In [ ]:
intersectionlist
fulllist = 1:size1
remaininglist = setdiff(fulllist, intersectionlist)

In [ ]:
c = 343 # speed of sound m/s (20degC)
duration = 0.1  # seconds
refl_coeff = 0.9  # reflection coefficient

draw = true  # to plot or not
apply_rigid = true  # apply rigid boundaries
apply_loss = true  # apply loss 

dx=0.1
dt = sqrt(1/3)*dx/c
SR = 1/dt
println("sample rate = ", SR)
println("Δx = $dx m")

Nx = size(grid)[1]+2  # number of points in x-dir
Ny = size(grid)[2]+2  # number of points in y-dir
Nz = size(grid)[3]+2  # number of points in y-dir
Nt = Int(ceil(duration/dt))  # number of time-steps

A = falses(Nx-2, Ny-2, Nz-2)
A[remaininglist] .=true
zeros_start = falses(1, Ny-2, Nz-2)
zeros_end = falses(1, Ny-2, Nz-2)
A_extended_x = cat(zeros_start, A, zeros_end, dims=1)
zeros_start = falses(Nx, 1, Nz-2)
zeros_end = falses(Nx, 1, Nz-2)
A_extended_y = cat(zeros_start, A_extended_x, zeros_end, dims=2)
zeros_start = falses(Nx, Ny, 1)
zeros_end = falses(Nx, Ny, 1)

in_mask = cat(zeros_start, A_extended_y, zeros_end, dims=3) #region in the boundary

In [ ]:
heatmap(transpose(in_mask[:, 5, : ]), color=:viridis, aspect_ratio=:equal, colorbar_title="Mask value")
title!("Mask XZ")
savefig("Mask XZ")

In [ ]:
heatmap(transpose(in_mask[:, :, 10 ]), color=:viridis, aspect_ratio=:equal, colorbar_title="Mask value")
title!("Mask XY")
savefig("Mask XY")

In [ ]:
heatmap(transpose(in_mask[40, :, : ]), color=:viridis, aspect_ratio=:equal, colorbar_title="Mask value")
title!("Mask YZ")
savefig("Mask YZ")

In [ ]:
if apply_rigid
    # calculate number of interior neighbours (for interior points only)
    K_map = zeros((Nx,Ny,Nz))
    K_map[2:Nx-1, 2:Ny-1, 2:Nz-1] .= in_mask[3:Nx, 2:Ny-1, 2:Nz-1] .+
                                    in_mask[1:Nx-2, 2:Ny-1, 2:Nz-1] .+
                                    in_mask[2:Nx-1, 3:Ny, 2:Nz-1] .+
                                    in_mask[2:Nx-1, 1:Ny-2, 2:Nz-1] .+
                                    in_mask[2:Nx-1, 2:Ny-1, 3:Nz] .+
                                    in_mask[2:Nx-1, 2:Ny-1, 1:Nz-2]
    K_map[.!in_mask] .= 0
    ib = findall((0 .< K_map .< 6))
    Kib = K_map[ib]
    K_map = nothing  # delete K_map
end

In [ ]:
# Initialize state arrays
u0 = zeros((Nx, Ny, Nz))
u1 = zeros((Nx, Ny, Nz))
u2 = zeros((Nx, Ny, Nz))
u3 = zeros((Nx, Ny, Nz))

# set up an excitation signal
u_in = zeros(Nt)
Nh = ceil(Int, 5 * SR / 500)
u_in[1:Nh] .= 0.5 .- 0.5 * cos.(2 * π * collect(1:Nh) / Nh)
u_in[1:Nh] .= u_in[1:Nh] .* sin.(2 * π * collect(1:Nh) / Nh)

# grid forcing points
inx = Int(ceil(Nx/2))
iny = Int(ceil(Ny/3))
inz = Int(ceil(Nz/2))
@assert in_mask[inx, iny, inz] "Invalid mask"

In [ ]:
if apply_loss
    # calculate specific admittance γ (g)
    @assert abs(refl_coeff) <= 1.0 "abs(refl_coeff) must be <= 1.0"
    g = (1 - refl_coeff) / (1 + refl_coeff)
    lf = 0.5 * √(0.5) * g  # a loss factor (?)
end


In [ ]:
output_folder = "output_images_julia"
if !isdir(output_folder)
    mkdir(output_folder)
end

tt = time()

bb = 0

for nt in 1:(Nt+2)
    # FDTD update
    u0[2:end-1, 2:end-1, 2:end-1] .= in_mask[2:end-1, 2:end-1, 2:end-1] .* ((u1[3:end, 2:end-1, 2:end-1] .+ u1[1:end-2, 2:end-1, 2:end-1] .+ u1[2:end-1, 3:end, 2:end-1] .+ u1[2:end-1, 1:end-2, 2:end-1] .+ u1[2:end-1, 2:end-1, 3:end] .+ u1[2:end-1, 2:end-1, 1:end-2]) ./ 3 .- u2[2:end-1, 2:end-1, 2:end-1])

    if apply_rigid
        ib_multi_indices = LinearIndices(u0)[ib]
        right = circshift(u1, (-1, 0, 0))[ib_multi_indices]
        left = circshift(u1, (1, 0, 0))[ib_multi_indices]
        forward = circshift(u1, (0, -1, 0))[ib_multi_indices]
        backward = circshift(u1, (0, 1, 0))[ib_multi_indices]
        up = circshift(u1, (0, 0, -1))[ib_multi_indices]
        down = circshift(u1, (0, 0, 1))[ib_multi_indices]

        u0[ib_multi_indices] .= (2 .- Kib ./ 3) .* u1[ib_multi_indices] .+ (right .+ left .+ forward .+ backward .+ up .+ down) ./ 3 .- u2[ib_multi_indices]


        if apply_loss
            u0[ib_multi_indices] .= (u0[ib_multi_indices] .+ lf .* (6 .- Kib) .* u2[ib_multi_indices]) ./ (1 .+ lf .* (6 .- Kib))
        end
    end

    if nt < length(u_in)
        u0[inx, iny, inz] += u_in[nt + 1]
    end

    if draw
        plot_data = copy(u0[:, iny, :])
        draw_mask = in_mask[:, iny, :]
        plot_data[.!draw_mask] .= NaN
        heatmap(transpose(plot_data), clim=(minimum(u0), maximum(u0)), color=:viridis, aspect_ratio=:equal, colorbar=true)
        title!("t = $(nt * dt) s")
        image_filename = joinpath(output_folder, "$(nt).png")
        savefig(image_filename)
    end

    u2, u1, u0 = u1, u0, u2
    if nt%100==0
        println(nt)
    end
end

tr = time() - tt
pstr = "Progress: finished"


In [ ]:
output_folder = "output_images_julia_fix"
if !isdir(output_folder)
    mkdir(output_folder)
end

tt = time()

bb = 0

for nt in 1:(Nt+2)
    # FDTD update
    u0[2:end-1, 2:end-1, 2:end-1] .= in_mask[2:end-1, 2:end-1, 2:end-1] .* ((u1[3:end, 2:end-1, 2:end-1] .+ u1[1:end-2, 2:end-1, 2:end-1] .+ u1[2:end-1, 3:end, 2:end-1] .+ u1[2:end-1, 1:end-2, 2:end-1] .+ u1[2:end-1, 2:end-1, 3:end] .+ u1[2:end-1, 2:end-1, 1:end-2]) ./ 3 .- u2[2:end-1, 2:end-1, 2:end-1])

    if apply_rigid
        ib_multi_indices = LinearIndices(u0)[ib]
        right = circshift(u1, (-1, 0, 0))[ib_multi_indices]
        left = circshift(u1, (1, 0, 0))[ib_multi_indices]
        forward = circshift(u1, (0, -1, 0))[ib_multi_indices]
        backward = circshift(u1, (0, 1, 0))[ib_multi_indices]
        up = circshift(u1, (0, 0, -1))[ib_multi_indices]
        down = circshift(u1, (0, 0, 1))[ib_multi_indices]

        u0[ib_multi_indices] .= (2 .- Kib ./ 3) .* u1[ib_multi_indices] .+ (right .+ left .+ forward .+ backward .+ up .+ down) ./ 3 .- u2[ib_multi_indices]


        if apply_loss
            u0[ib_multi_indices] .= (u0[ib_multi_indices] .+ lf .* (6 .- Kib) .* u2[ib_multi_indices]) ./ (1 .+ lf .* (6 .- Kib))
        end
    end
    
    if nt < length(u_in)
        u0[inx, iny, inz] += u_in[nt + 1]
    end

    if draw
        plot_data = copy(u0[:, iny, :])
        draw_mask = in_mask[:, iny, :]
        plot_data[.!draw_mask] .= NaN
        heatmap(transpose(plot_data), clim=(-0.03, 0.03), color=:viridis, aspect_ratio=:equal, colorbar=true)
        title!("t = $(nt * dt) s")
        image_filename = joinpath(output_folder, "$(nt).png")
        savefig(image_filename)
    end
    
    u2, u1, u0 = u1, u0, u2
    if nt%100==0
        println(nt)
    end
end

tr = time() - tt
pstr = "Progress: finished"